In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup
from datasets import Dataset

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Loading and cleaning the data

In [3]:
classified_data = pd.read_excel('C:/Users/jaoss/Desktop/Data_Science/Master_Project/Git_Repository/ECB_Perceived_Cacophony/classified_articles_50.xlsx')
classified_data.head()

,Date,Source,Headline,Translated.headline,Manual.summary,Original.article.url,Translated.text,Media.type,Speaker,Reach,Language,Country.Code,Sitename,Tags,Site_Readership,Article_Readership,Classification,Score,Explanation
0,2022-09-30 22:08:34,Il Messaggero - Borsa Italiana,"Visco: «Troppi rialzi dei tassi, si va in rece...","Banca d'Italia's Visco: ""Too many rate hikes w...","At a conference in Florence, Banca d'Italia Go...",https://www.ilmessaggero.it/economia/news/tass...,No to an excessive rise in rates. The Governor...,WEB,Newspapers,1174293,it,IT,Il Messaggero,"European Central Bank, European Monetary Union...",2296944.0,0.0,Dovish,-0.6,The article leans towards a dovish sentiment a...
1,2022-09-30 21:30:13,Handelsblatt - Finanzen,EZB-Ratsmitglied Visco: Rezessionsrisiko bei g...,ECB 's Visco: Recession risk in the event of m...,Banca d'Italia Governor Ignazio Visco on Fri...,https://www.handelsblatt.com/finanzen/geldpoli...,NaN,WEB,Newspapers,525924,de,DE,Handelsblatt,"European Central Bank, European Monetary Union...",314148.0,811493.0,Dovish,-0.8,The article suggests a dovish sentiment by adv...
2,2022-09-30 20:56:08,Bloomberg - GNews,ECB 's Schnabel Says Weaker Demand May Not Eas...,ECB 's Schnabel says weaker demand may not eas...,ECB Executive Board Member Isabel Schnabel ...,https://www.bloomberg.com/news/articles/2022-0...,European Central Bank Executive Board member I...,WEB,Net Magazine,2520979,en,US,Bloomberg,"European Central Bank, Inflation, Rank 1, Core...",1309524.0,278016.0,Hawkish,0.8,The article suggests a hawkish stance by advoc...
3,2022-09-30 18:42:47,Il Foglio - Page 3,"**Conti pubblici: Visco, rientro debito ben in...","Public accounts: Visco, debt repayment well ad...",In a speech organised by the Florence School o...,https://www.ilfoglio.it/adnkronos/2022/09/30/n...,NaN,WEB,Newspapers,230279,it,IT,Il Foglio,"European Central Bank, European Monetary Union...",204119.0,0.0,Dovish,-0.6,The article suggests a dovish sentiment as it ...
4,2022-09-30 18:32:37,Finance.si - Live,Evrska inflacija na novem rekordu. Bo ECB ok...,Euro area inflation on a new record. Will the ...,"According to first estimates, euro area inflat...",https://www.finance.si/9004503/Evrska-inflacij...,NaN,WEB,Net Magazine,31368,sl,SI,Finance.si,"European Central Bank, Inflation, Rank 2, Chri...",179079.0,0.0,Hawkish,0.8,The article suggests concerns about high infla...


In [4]:
classified_data['Manual.summary'].tolist()[4]

'According to first estimates, euro area inflation jumped to 10% in September, which is a new record for the region. In August, euro area inflation, as measured by the harmonised consumer price index, was 9.1%, and last September it was 3.4%. Record price growth and the fact that inflation is now expressed in double digits in Germany mean additional pressure on the  ECB  to take measures to control it.  ECB  Governing Council members will meet on 27 October and many experts consider that the  central bank  should continue to be aggressive in the future.  ECB  President  Christine Lagarde  has explicitly confirmed that interest rate hikes are far from over.'

In [5]:
total_data = pd.read_excel('C:/Users/jaoss/Desktop/Data_Science/Master_Project/combined.xlsx')
total_data.drop_duplicates(subset=['Manual.summary'], inplace=True)
total_data.dropna(subset='Manual.summary', inplace=True)
print(total_data.shape)
total_data.head()

(29823, 15)


,Date,Source,Headline,Translated.headline,Manual.summary,Original.article.url,Media.type,Speaker,Reach,Language,Country.Code,Sitename,Tags,Site_Readership,Article_Readership
0,2022-09-30 22:08:34,Il Messaggero - Borsa Italiana,"Visco: «Troppi rialzi dei tassi, si va in rece...","Banca d'Italia's Visco: ""Too many rate hikes w...","At a conference in Florence, Banca d'Italia Go...",https://www.ilmessaggero.it/economia/news/tass...,WEB,Newspapers,1174293,it,IT,Il Messaggero,"European Central Bank, European Monetary Union...",2296944.0,0.0
2,2022-09-30 21:30:13,Handelsblatt - Finanzen,EZB-Ratsmitglied Visco: Rezessionsrisiko bei g...,ECB 's Visco: Recession risk in the event of m...,Banca d'Italia Governor Ignazio Visco on Fri...,https://www.handelsblatt.com/finanzen/geldpoli...,WEB,Newspapers,525924,de,DE,Handelsblatt,"European Central Bank, European Monetary Union...",314148.0,811493.0
4,2022-09-30 20:56:08,Bloomberg - GNews,ECB 's Schnabel Says Weaker Demand May Not Eas...,ECB 's Schnabel says weaker demand may not eas...,ECB Executive Board Member Isabel Schnabel ...,https://www.bloomberg.com/news/articles/2022-0...,WEB,Net Magazine,2520979,en,US,Bloomberg,"European Central Bank, Inflation, Rank 1, Core...",1309524.0,278016.0
11,2022-09-30 18:42:47,Il Foglio - Page 3,"**Conti pubblici: Visco, rientro debito ben in...","Public accounts: Visco, debt repayment well ad...",In a speech organised by the Florence School o...,https://www.ilfoglio.it/adnkronos/2022/09/30/n...,WEB,Newspapers,230279,it,IT,Il Foglio,"European Central Bank, European Monetary Union...",204119.0,0.0
13,2022-09-30 18:32:37,Finance.si - Live,Evrska inflacija na novem rekordu. Bo ECB ok...,Euro area inflation on a new record. Will the ...,"According to first estimates, euro area inflat...",https://www.finance.si/9004503/Evrska-inflacij...,WEB,Net Magazine,31368,sl,SI,Finance.si,"European Central Bank, Inflation, Rank 2, Chri...",179079.0,0.0


In [6]:
# Drop rows with NaN values in 'Manual.summary' column from total_data
total_data_cleaned = total_data.dropna(subset=['Manual.summary'])['Manual.summary']

# Select desired columns from classified_data
classified_data_selected = classified_data[['Manual.summary', 'Classification']]

# Concatenate the dataframes
data = pd.concat([total_data_cleaned, classified_data_selected])
data.drop_duplicates(subset='Manual.summary', inplace=True)

In [7]:
# Select a random subset of the data that have NaN values in Classification of 450 rows
data = pd.concat([data[data['Classification'].isnull()].sample(n=0, random_state=28), classified_data_selected])
data.reset_index(drop=True, inplace=True)
print(data.shape)
data.head()

(50, 2)


,Manual.summary,Classification
0,"At a conference in Florence, Banca d'Italia Go...",Dovish
1,Banca d'Italia Governor Ignazio Visco on Fri...,Dovish
2,ECB Executive Board Member Isabel Schnabel ...,Hawkish
3,In a speech organised by the Florence School o...,Dovish
4,"According to first estimates, euro area inflat...",Hawkish


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Manual.summary  50 non-null     object
 1   Classification  50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes


## Model
### Pretrained BERT Model

In [9]:
# Load the BERT tokenizer
model_name = "gtfintechlab/FOMC-RoBERTa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

In [10]:
# Change column name to text and the classifications to 1 for hawkish and 0 for dovish
data.rename(columns={'Manual.summary': 'text'}, inplace=True)
data.rename(columns={'Classification': 'label'}, inplace=True)
data['label'] = data['label'].map({'Hawkish': 1, 'Dovish': 0, pd.NA: np.nan})

# Change type to int
data['label'] = data['label'].astype(int)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(data)

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [12]:
# Split dataset: use 40 for training, 10 for validation
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(40))
small_eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(40, 50))

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

c:\Users\jaoss\Desktop\Data_Science\mycondaenv\Lib\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [14]:
trainer.train()

  0%|          | 0/15 [00:00<?, ?it/s]

In [23]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

tokenizer = AutoTokenizer.from_pretrained("gtfintechlab/FOMC-RoBERTa", do_lower_case=True, do_basic_tokenize=True)
model = AutoModelForSequenceClassification.from_pretrained("gtfintechlab/FOMC-RoBERTa", num_labels=3)
config = AutoConfig.from_pretrained("gtfintechlab/FOMC-RoBERTa")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, config=config, device=-1, framework="pt")
results = classifier(["Such a directive would imply that any tightening should be implemented promptly if developments were perceived as pointing to rising inflation.", 
                    "The International Monetary Fund projects that global economic growth in 2019 will be the slowest since the financial crisis."], 
                    batch_size=128, truncation="only_first")

print(results)


[{'label': 'LABEL_1', 'score': 0.999393105506897}, {'label': 'LABEL_0', 'score': 0.9979877471923828}]


In [25]:
import pandas as pd
from datasets import Dataset

# Load your labeled dataset (50 labeled articles)
labeled_df = pd.read_csv('C:/Users/jaoss/Desktop/Data_Science/Master_Project/Git_Repository/ECB_Perceived_Cacophony/labeled_articles.csv')

# Ensure the labels are integers
labeled_df['label'] = labeled_df['label'].astype(int)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(labeled_df)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gtfintechlab/FOMC-RoBERTa")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])  # Remove raw text column
tokenized_datasets.set_format("torch")  # Set format to PyTorch tensors